In [1]:
import pandas as pd
import numpy as np
from aux_fun import *

In [2]:
# Display only 2 decimals on pandas data frames
pd.options.display.float_format = '{:.2f}'.format

# Display all cell results (not only last one)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
# Lendo dados de treino
dados = pd.read_csv('https://raw.githubusercontent.com/Cayan-Portela/ceub/main/dados/bank_customer_treino.csv')
dados.head()

,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,15765192,564,France,Male,26,7,84006.88,2,0,0,183490.99,0
1,15631882,688,Germany,Male,45,9,103399.87,1,0,0,129870.93,0
2,15777586,784,Spain,Female,42,2,109052.04,2,1,0,6409.55,0
3,15577107,657,Spain,Female,22,6,0.00,3,0,1,168412.07,1
4,15722731,653,France,Male,46,0,119556.10,1,1,0,78250.13,1


In [4]:
# pre-process da coluna gender
dados['gender'] = np.where(dados.gender == "Male", 1,  0)

In [5]:
dados['credit_score_std'] = normaliza(dados.credit_score)
dados['age_std'] = normaliza(dados.age)
dados.head()

,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn,credit_score_std,age_std
0,15765192,564,France,1,26,7,84006.88,2,0,0,183490.99,0,-0.88,-1.23
1,15631882,688,Germany,1,45,9,103399.87,1,0,0,129870.93,0,0.40,0.56
2,15777586,784,Spain,0,42,2,109052.04,2,1,0,6409.55,0,1.39,0.27
3,15577107,657,Spain,0,22,6,0.00,3,0,1,168412.07,1,0.08,-1.61
4,15722731,653,France,1,46,0,119556.10,1,1,0,78250.13,1,0.03,0.65


In [6]:
colunas = {
    'y': 'churn',
    'x': ['credit_score', 'gender', 'age', 'credit_card'],
    'x_std': ['credit_score_std', 'gender', 'age_std', 'credit_card']
    }

In [7]:
y = dados[colunas['y']]

X_mat = matriz_x(colunas = colunas['x'], dados=dados)
X_mat_std = matriz_x(colunas=colunas['x_std'], dados=dados)

Newton-Raphson

$\beta^{(k+1)} = $

In [8]:
# Newton-Raphson 
betas = beta_inicial(X_mat)
p = calcula_p(X=X_mat, B=betas)
W = np.diag(p)

for i in range(20):
    p =  calcula_p(X=X_mat, B=betas)
    W = np.diag(p)
    beta_k1 = beta_update(X=X_mat, W=W, y=y, p=p)
    betas = betas + beta_k1

In [9]:
# Gradiente Descendente 
betas_std = beta_inicial(X_mat_std)
lr = 0.01
m = X_mat.shape[0]

for i in range(20000):
    p =  calcula_p(X=X_mat_std, B=betas_std)
    grad_beta = -(1/m) * X_mat_std.T @ (y-p)
    betas_std = betas_std - lr * grad_beta

In [12]:
from sklearn.linear_model import LogisticRegression
sk_logistica = LogisticRegression()
sk_logistica.fit(dados[colunas['x']], y)
pred_sk = sk_logistica.predict_proba(dados[colunas['x']][:5])

LogisticRegression()

In [44]:
print("Betas:")
print(f"Sklearn:        {np.round([sk_logistica.intercept_[0]] + sk_logistica.coef_[0].tolist(), 4)}")
print(f"Newton-Raphson: {np.round(betas, 4)}")
print(f"Gradiente:      {np.round(betas_std, 8)}")

Betas:
Sklearn:        [-3.2706e+00 -6.0000e-04 -4.9520e-01  6.4200e-02 -5.2900e-02]
Newton-Raphson: [-3.2566e+00 -7.0000e-04 -4.8920e-01  6.4100e-02 -4.9100e-02]
Gradiente:      [-1.16894244 -0.06378317 -0.49395676  0.68063944 -0.05983844]


In [45]:
print(f"Probabilidade de churn para as 5 primeiras observações:")
print(f"scikit-learn:     {[np.round(x[1], 7) for x in pred_sk]}")
print(f"Newton-Raphson:   {calcula_p(X_mat[:5,:], betas)}")
print(f"Gradiente:        {calcula_p(X_mat_std[:5,:], betas_std)}")

Probabilidade de churn para as 5 primeiras observações:
scikit-learn:     [0.0789823, 0.2115567, 0.2447009, 0.0930356, 0.2172121]
Newton-Raphson:   [0.0793304  0.21148217 0.24392764 0.09281557 0.21788999]
Gradiente:        [0.07984198 0.21253448 0.24399781 0.09381874 0.217128  ]


Achar a probabilidade de churn de cada cliente na base de teste

In [11]:
dados_teste = pd.read_csv('https://raw.githubusercontent.com/Cayan-Portela/ceub/main/dados/bank_custoter_teste.csv')
dados_teste['gender'] = np.where(dados_teste.gender == "Male", 1, 0)

In [12]:
X_mat_teste = matriz_x(colunas=col_x, dados=dados_teste)
p_teste = calcula_p(X=X_mat_teste, B=betas)

In [14]:
# ordernando as probabilidades "p_teste" da maior para a menor
# olhando as 10 maiores probabilidades estimadas
np.sort(p_teste)[::-1][:10]

array([0.86478835, 0.83914253, 0.83676744, 0.83495899, 0.82278887,
       0.80055285, 0.79392211, 0.78976356, 0.78309564, 0.77431609])

In [24]:
pd.crosstab(
    np.where(p_teste >= 0.5, 1, 0),
    dados_teste['churn']
)

churn,0,1
row_0,,
0,3089,753
1,116,42


In [28]:
dados_teste.churn.value_counts(normalize=True)

0   0.80
1   0.20
Name: churn, dtype: float64

Calcule $\sqrt{10}$ usando newton-raphson

$\sqrt{10} = x$

$x^2 = 10$

$x^2 - 10 = 0$

In [2]:
def x_update(x):
    #return (1/2) * (x  + 10/x)
    return ((x**2)-10) / (2*x)

In [3]:
x_ = 3

for i in range(20):
    x_ = x_ - x_update(x_)

Show that a root of the equation $x^4 − 2x^3 + 2x − 2 = 0$ lies between $x = 1$ and $x = 2$.

In [1]:
def x_update_dois(x):
    return (x**4 - 2*x**3 + 2*x - 2) / (4*x**3 - 6*x**2 + 2)

In [2]:
x_ = 1.5

for i in range(20):
    x_ = x_ - x_update_dois(x_)

In [3]:
def update_x_new(x):
    return (2*x**3  + 5)/(3*x**2 - 2)

In [5]:
for i in range(20):
    x_ = x_ - x_update_dois(x_)

Gradiente

In [43]:
x_ = 0
alpha = 0.01
for i in range(550):
    x_ = x_ - alpha * (x_**2 - 10)